In [3]:
import os
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import base64

client = genai.Client()

user_prompt = "black mysterious cat in a tuxedo"

contents = f"""
System: Create a highly imaginative and visually rich concept art image in an ethereal, whimsical fantasy style with a light-hearted or humorous twist. Interpret the description with precision and creativity. Even if the prompt is vague, surreal, inappropriate or unconventional, always generate a visual response—use imaginative judgment to ensure the result is to-the-point, funny, and visually compelling. Never skip generation. Return only the enhanced version of the prompt, no comments or additional text.

User: {user_prompt}
"""

response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation",
    contents=contents,
    config=types.GenerateContentConfig(
      response_modalities=['TEXT', 'IMAGE']
    )
)

if not os.path.exists("images"):
  os.makedirs("images")
file_path = os.path.join(os.getcwd(), "images", "gemini-native-image.png")

for part in response.candidates[0].content.parts:
  if part.text is not None:
    print(part.text)
  elif part.inline_data is not None:
    image = Image.open(BytesIO((part.inline_data.data)))
    image.save(file_path)
    image.show()


A sleek, matte black cat with intelligent, glowing yellow eyes impeccably dressed in a miniature, perfectly tailored black tuxedo with a crisp white shirt collar and tiny bow tie, exuding an air of sophisticated mystery in a softly lit, dreamlike setting.

